In [ ]:
import os
import urllib.request
import json
import openai
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_TOKEN")

In [ ]:
import replicate
text2image = replicate.models.get("stability-ai/stable-diffusion")
text2image_version = text2image.versions.get("f178fa7a1ae43a9a9af01b833b9d2ecf97b1bcb0acfd2dc5dd04895e042863f1")

image2image = replicate.models.get("stability-ai/stable-diffusion-img2img")
image2image_version = image2image.versions.get("15a3689ee13b0d2616e98820eca31d4c3abcd36672df6afce5cb6feb1d66087d")

image2text = replicate.models.get("pharmapsychotic/clip-interrogator")
image2text_version = image2text.versions.get("a4a8bafd6089e1716b06057c42b19378250d008b80fe87caa5cd36d40c1eda90")

## Constants

In [ ]:
IMAGE_SAVE_PATH='/var/www/output.designresearch.works/'

### Endpoints: /heartbeat

#### Input:
None

#### Output:
{ \
    "status": string \
}

In [ ]:
# GET /heartbeat
print(json.dumps({'status': 'alive'}))

In [ ]:
# ResponseInfo GET /heartbeat
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

### Endpoints: /text_to_text

#### Input:
{ \
    "prompt": string, \
    "max_tokens": int (default: 100), \
    "temperature": float (defaut: 0.0) \
} 

#### Output:
{ \
    "completion": string \
}

In [ ]:
# GET /text_to_text
req = json.loads(REQUEST)
body = req['body']

completion_object = openai.Completion.create(
  model="text-davinci-003",
  prompt=body['prompt'],
  max_tokens=body['max_tokens'] if 'max_tokens' in body else 100,
  temperature=body['temperature'] if 'temperature' in body else 0
)

print(json.dumps({
    'completion': completion_object.choices[0].text
}))

In [ ]:
# ResponseInfo GET /text_to_text
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

### Endpoints: /text_to_image

#### Input:
{ \
    "prompt": string, \
    "cfg_scale": float, \
    "init_img_url": string (URL of the image, default: None), \
    "prompt_strength": float (default: 1.0), \
    "num_outputs": int (default: 1) \
} 

#### Output:
{ \
    "image_url": lstring \
}

In [ ]:
# GET /text_to_image
req = json.loads(REQUEST)
body = req['body']

output = text2image_version.predict(
    prompt=body['prompt'],
    guidance_scale=body['cfg_scale'],
    init_image=body['init_img_url'] if 'init_img_url' in body else None,
    prompt_strength=body['prompt_strength'] if 'prompt_strength' in body else 1.0,
    num_outputs=int(body['num_outputs']) if 'num_outputs' in body and body['num_outputs'] > 0 else 1
)

# save images locally
for url in output:
    urllib.request.urlretrieve(url, IMAGE_SAVE_PATH + "_".join(url.split('/')[-2:]))

print(json.dumps({
    'image_url': output
}))

In [ ]:
# ResponseInfo GET /text_to_image
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

### Endpoints: /image_to_image

#### Input:
{ \
    "image_url": string (URL of the image), \
    "prompt": string, \
    "cfg_scale": float, \
    "prompt_strength": float (default: 1.0), \
    "num_outputs": int (default: 1) \
} 

#### Output:
{ \
    "image_url": string \
}

In [ ]:
# GET /image_to_image
req = json.loads(REQUEST)
body = req['body']

output = image2image_version.predict(
    image=body['image_url'],
    prompt=body['prompt'],
    guidance_scale=body['cfg_scale'],
    prompt_strength=body['prompt_strength'] if 'prompt_strength' in body else 1.0,
    num_outputs=int(body['num_outputs']) if 'num_outputs' in body and body['num_outputs'] > 0 else 1
)

# save images locally
for url in output:
    urllib.request.urlretrieve(url, IMAGE_SAVE_PATH + "_".join(url.split('/')[-2:]))

print(json.dumps({
    'image_url': output
}))

In [ ]:
# ResponseInfo GET /image_to_image
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

### Endpoints: /image_to_text

#### Input:
{ \
    "image_url": string (URL of the image), \
} 

#### Output:
{ \
    "image_url": string \
}

In [ ]:
# GET /image_to_text
req = json.loads(REQUEST)
body = req['body']

output = image2text_version.predict(
    image=body['image_url'],
    clip_model_name='ViT-H-14/laion2b_s32b_b79k',
    mode='fast'
)

print(json.dumps({
    'image_url': output
}))

In [ ]:
# ResponseInfo GET /image_to_text
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))